In [37]:
!pip install --upgrade simpy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Voglio modellare un solo pullman con 10 posti e clienti che arrivano ogni 1 tu.

1) Il bus è in attesa e appena salgono 10 cienti parte.
2) impiega 12 tu ad andare/scaricare e tornare.
3) Carica altri 10 clienti e riparte.

In [38]:
import simpy

In [39]:
class Bus(object):
    def __init__(self, env, bus_capacity, travel_time=12):
        self.env = env
        self.travel_time = travel_time
        self.seats = simpy.Container(env, capacity=bus_capacity, init=0)
        self.departure_event = env.event()
        self.arrival = env.event()
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{self.env.now:.2f}: Bus alla fermata in attesa del segnale di partenza.')
            
            # 1. ATTENDI IL SEGNALE
            yield self.departure_event
            
            # 2. PARTI E VIAGGIA
            print(f'{self.env.now:.2f}: Segnale ricevuto! Bus parte con {self.seats.level} passeggeri.')

            # Dopo metà del tempo di viaggio, il bus arriva a destinazione
            # e segnala il suo arrivo.
            yield self.env.timeout(self.travel_time/2)
            self.arrival.succeed()

            # Viaggio di ritorno
            yield self.env.timeout(self.travel_time/2)

            # 3. TORNA E SVUOTA I POSTI
            print(f'{self.env.now:.2f}: Bus tornato.')
            yield self.seats.get(self.seats.capacity)
            # segnala che il bus è tornato e 
            
            # 4. PREPARA IL PROSSIMO CICLO
            self.arrival = self.env.event()
            self.departure_event = self.env.event()

In [40]:
env = simpy.Environment()
# Create a bus with a capacity of 10 passengers
bus = Bus(env,10)

In [41]:
# --- Processo Passeggero (con controllo di sicurezza sull'evento) ---
def passeggero(env, nome, bus, attese):
    print(f'{env.now:.2f}: Il passeggero {nome} arriva alla fermata.')
    arrivo = env.now
    yield bus.seats.put(1)
    salito = env.now
    print(f'{env.now:.2f}: Il passeggero {nome} è salito. Posti occupati: {bus.seats.level}')
    
    # Se il bus è pieno E l'evento di partenza non è già stato attivato...
    if bus.seats.level == bus.seats.capacity and not bus.departure_event.triggered:
        # ...allora attiva l'evento.
        print(f'{env.now:.2f}: Bus pieno! Il passeggero {nome} segnala la partenza.')
        bus.departure_event.succeed()
    yield bus.arrival  # Attendi l'arrivo del bus
    discesa = env.now
    attesa = discesa - arrivo
    attese.append(attesa)

# --- Processo Generatore di Passeggeri (invariato) ---
def arrivi_passeggeri(env, bus, attese):
    contatore_passeggeri = 0
    while True:
        yield env.timeout(1)
        contatore_passeggeri += 1
        env.process(passeggero(env, f'P{contatore_passeggeri}', bus, attese))

In [42]:
attese = []
passenger_arrival_process = env.process(arrivi_passeggeri(env, bus, attese))
env.run(until=200)  # Run the simulation for 50 time units


0.00: Bus alla fermata in attesa del segnale di partenza.
1.00: Il passeggero P1 arriva alla fermata.
1.00: Il passeggero P1 è salito. Posti occupati: 1
2.00: Il passeggero P2 arriva alla fermata.
2.00: Il passeggero P2 è salito. Posti occupati: 2
3.00: Il passeggero P3 arriva alla fermata.
3.00: Il passeggero P3 è salito. Posti occupati: 3
4.00: Il passeggero P4 arriva alla fermata.
4.00: Il passeggero P4 è salito. Posti occupati: 4
5.00: Il passeggero P5 arriva alla fermata.
5.00: Il passeggero P5 è salito. Posti occupati: 5
6.00: Il passeggero P6 arriva alla fermata.
6.00: Il passeggero P6 è salito. Posti occupati: 6
7.00: Il passeggero P7 arriva alla fermata.
7.00: Il passeggero P7 è salito. Posti occupati: 7
8.00: Il passeggero P8 arriva alla fermata.
8.00: Il passeggero P8 è salito. Posti occupati: 8
9.00: Il passeggero P9 arriva alla fermata.
9.00: Il passeggero P9 è salito. Posti occupati: 9
10.00: Il passeggero P10 arriva alla fermata.
10.00: Il passeggero P10 è salito. Posti 